In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import random
import tensorflow as tf
from model_structures import *
from data_loader import *
from loss_func import *
from QSM_func import *
from keras.optimizers import Adam
import datetime
import time
tf.__version__

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
opt = {}

opt['train_path'] = '/home/sylar/data/Simu/dp_inv_simu_train/simu_new_dipole/simu_111'
opt['train_img1_path'] = ['/phs/phs']
opt['train_img2_path'] = ['/phs/phs']
opt['train_label_path'] = ['/qsm/qsm']

opt['reso'] = (1,1,1)
opt['patch_size'] = (64,64,64)

opt['lbd2'] = 0.1
opt['iter'] = 5
opt['batch_size']= 15
opt['channels'] = 1
opt['img_shape'] = opt['patch_size'] + (opt['channels'],)
opt['in_shape'] = opt['patch_size'] + (1,)
opt['rnd_crop'] = False
opt['is_aug'] = True

opt['model_restored'] = False
opt['model_restored_epoch'] = 95 # from 35 add random combine   // from 44 to nrmse
opt['model_total_epoch'] = 200
opt['model_save_interval'] = 5

opt['learning_rate'] = 2e-5
opt['beta_1'] = 0.9
opt['beta_2'] = 0.99

opt['loss'] = nrmse
opt['display_nums'] = [50,20,30]

opt['model_save_path'] = '/home/sylar/data/results/model1_revision/pocsnet2_model'
opt['checkpoint_path'] = opt['model_save_path']+"/cp{epoch}"

In [ ]:
if opt['model_restored'] == False:
    index = list(range(1,1001))
    random.shuffle(index)
    opt['train_index'] = index[0:950]
    opt['val_index'] = index[950:1000]
    np.save(opt['model_save_path'] + '/train_index8.npy', opt['train_index'])
    np.save(opt['model_save_path'] + '/val_index2.npy', opt['val_index'])
else:
    opt['train_index'] = np.load(opt['model_save_path']+'/train_index8.npy')
    opt['val_index'] = np.load(opt['model_save_path']+'/val_index2.npy')

In [ ]:
opt['train_data'] = Data_loaders_dpinv(opt)
x1_train, x2_train, y_train = opt['train_data'].next([1],opt)
print(opt['train_data'].data_size)

In [ ]:
'''loading data preshow'''
ch,cw,cd = findcenter3d(y_train[0])
aa=cw
dd=cd

f, axarr = plt.subplots(2, 3, figsize=(20, 10))
axarr[1,0].imshow(np.flip(np.transpose(x1_train[0,:,aa,:].squeeze()),0))
axarr[1,0].axis('off')
axarr[0,2].imshow(np.transpose(y_train[0,:,:,dd].squeeze()))
axarr[0,2].axis('off')
axarr[0,0].imshow(np.transpose(x1_train[0,:,:,dd].squeeze()))
axarr[0,0].axis('off')
axarr[1,1].imshow(np.flip(np.transpose(x2_train[0,:,aa,:].squeeze()),0))
axarr[1,1].axis('off')
axarr[1,2].imshow(np.flip(np.transpose(y_train[0,:,aa,:].squeeze()),0))
axarr[1,2].axis('off')
axarr[0,1].imshow(np.transpose(x2_train[0,:,:,dd].squeeze()))
axarr[0,1].axis('off')
plt.show()

In [ ]:
tf.compat.v1.disable_eager_execution()
D = dipole_kernel(opt)
model = {}

model['vnet'] = unet_at2(opt, 1)
x = Input(shape=opt['img_shape'])

x_k = phase2qsm(x,D,opt)
g = x_k

for it in range(opt['iter']):
    net_out = model['vnet'](x_k)

    #updata x_k, qsm
    x_k = g + opt['lbd2']*phaseinv(net_out,D,opt)

model['optimizer'] = Adam(opt['learning_rate'], opt['beta_1'], opt['beta_2'])

model['combine'] = Model(inputs = x,
                     outputs = [net_out, x_k],
#                      outputs = x_k,
                     name='combine_model')

model['combine'].compile(optimizer = model['optimizer'],
                     loss = [opt['loss'],opt['loss']],
                     loss_weights = [0.5, 0.5],
                     metrics=['accuracy'])

# Create a callback that saves the model's weights
if opt['model_restored'] == True:
    model['vnet'].load_weights(opt['checkpoint_path'].format(epoch = opt['model_restored_epoch']))
    start_epoch = opt['model_restored_epoch']
else:
    start_epoch = 1

In [ ]:
nr_train_im = opt['train_data'].data_size
nr_im_per_epoch = int(np.ceil(nr_train_im / opt['batch_size']) * opt['batch_size'])

start_time = time.time()
print("Start Time:" + str(start_time))

avg_epoch_cost = []
print('-----Start Training-----')
for epoch in range(start_epoch, opt['model_total_epoch']):
 
    order = np.concatenate((np.random.permutation(nr_train_im),
                                         np.random.randint(nr_train_im, size=nr_im_per_epoch - nr_train_im)))
    avg_img_cost = []
    for block_i in range(1, nr_im_per_epoch+1, opt['batch_size']):
        index = order[block_i:block_i+opt['batch_size']]
        x1_train, x2_train, y_train = opt['train_data'].next(index, opt)

        # Training
        x_in = x2_train
        Loss = model['combine'].train_on_batch(x = x_in, y = [y_train,y_train])
        [y_pred1, y_pred2] = model['combine'].predict(x_in)
        m_loss1 = Loss[0]
        m_loss2 = 0
        m_loss3 = 0
        norm_loss = np.linalg.norm(y_pred1-y_train)/np.linalg.norm(y_train)
        avg_img_cost.append(m_loss1)
        if block_i % (30 * opt['batch_size'])==1:
            # Plot the progress
            print ("[Epoch %d/%d] [Batch %d/%d] [Model loss: %f-%f-%f ; nrmse: %f]" % (epoch, opt['model_total_epoch'],
                                                                block_i, nr_train_im,
                                                                m_loss1,m_loss2,m_loss3, norm_loss))
            display_slice(opt['display_nums'], y_pred1,y_pred1, y_train)
    avg_epoch_cost.append(np.mean(avg_img_cost)) 
                                
    print("Epoch:", '%04d' % (epoch), "Training_cost=", "{:.5f}".format(avg_epoch_cost[-1]))
    display_error(range(start_epoch,epoch+1),avg_epoch_cost)

    # If at save interval => save models
    if epoch % opt['model_save_interval'] == 0:
        model['vnet'].save_weights(opt['checkpoint_path'].format(epoch=epoch))
            
elapsed_time = time.time() - start_time
print("Total Time:" + str(elapsed_time))

In [ ]:
# !nvidia-smi